# Setup

In [ ]:
pip install autofeat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 10.3 MB/s eta 0:00:00


In [ ]:
# import modules
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from autofeat import FeatureSelector, AutoFeatRegressor

import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [ ]:
# mounting Google Drive and setting path
from google.colab import drive
drive.mount('/content/drive')
path = 'drive/MyDrive/DS301 Final Project/data/'

Mounted at /content/drive


In [ ]:
# read TSLA dataset into a DataFrame 
names = ['t_date', 'stock_id', 'stock_symbol', 'expiration_date', 'strike', 
         'call_put', 'style', 'symbol', 'price_bid', 'price_ask', 'date_bid', 
         'date_ask', 'size_bid', 'size_ask','exchange_bid', 'exchange_ask', 'volume',
         'implied_volatility', 'price_opt', 'delta', 'gamma', 'theta', 'vega', 
         'rho', 'pre_iv', 'implied_yield', 'calc_date', 'dump_time']

tsla = pd.read_csv(path + "options_tsla.csv", names=names)

In [ ]:
tsla

,t_date,stock_id,stock_symbol,expiration_date,strike,call_put,style,symbol,price_bid,price_ask,date_bid,date_ask,size_bid,size_ask,exchange_bid,exchange_ask,volume,implied_volatility,price_opt,delta,gamma,theta,vega,rho,pre_iv,implied_yield,calc_date,dump_time
0,2021-10-19 09:30:00,21400,TSLA,2021-10-22 00:00:00,50.0,C,A,TSLA 211022C00050000,0.00,0.00,2021-10-19 09:24:59,2021-10-19 09:24:59,0,0,*,*,0,-1.0000,870.11,0.00000,0.000000,0.000000,0.00000,0.00000,-1.0000,0.0,2021-10-19 09:25:46,2021-10-19 09:30:00
1,2021-10-19 09:30:00,21400,TSLA,2021-10-22 00:00:00,50.0,P,A,TSLA 211022P00050000,0.00,0.00,2021-10-19 09:25:01,2021-10-19 09:25:01,0,0,*,*,0,-1.0000,870.11,0.00000,0.000000,0.000000,0.00000,0.00000,-1.0000,0.0,2021-10-19 09:25:46,2021-10-19 09:30:00
2,2021-10-19 09:30:00,21400,TSLA,2021-10-22 00:00:00,100.0,C,A,TSLA 211022C00100000,0.00,0.00,2021-10-19 09:25:07,2021-10-19 09:25:07,0,0,*,*,0,-1.0000,870.11,0.00000,0.000000,0.000000,0.00000,0.00000,-1.0000,0.0,2021-10-19 09:25:46,2021-10-19 09:30:00
3,2021-10-19 09:30:00,21400,TSLA,2021-10-22 00:00:00,100.0,P,A,TSLA 211022P00100000,0.00,0.00,2021-10-19 09:25:06,2021-10-19 09:25:06,0,0,*,*,0,-1.0000,870.11,-0.00000,0.000000,-0.000000,0.00000,-0.00000,-1.0000,0.0,2021-10-19 09:25:46,2021-10-19 09:30:00
4,2021-10-19 09:30:00,21400,TSLA,2021-10-22 00:00:00,150.0,C,A,TSLA 211022C00150000,0.00,0.00,2021-10-19 09:25:01,2021-10-19 09:25:01,0,0,*,*,0,-1.0000,870.11,0.00000,0.000000,0.000000,0.00000,0.00000,-1.0000,0.0,2021-10-19 09:25:46,2021-10-19 09:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000823,2021-10-19 16:18:00,21400,TSLA,2024-01-19 00:00:00,1675.0,P,A,TSLA 240119P01675000,857.00,874.05,2021-10-19 15:59:58,2021-10-19 15:59:58,9,95,AO,XO,2,0.4361,864.27,-0.75618,0.000577,-0.099708,4.09159,-18.16715,0.4361,0.0,2021-10-19 16:16:38,2021-10-19 16:18:00
2000824,2021-10-19 16:18:00,21400,TSLA,2024-01-19 00:00:00,1700.0,C,A,TSLA 240119C01700000,70.05,75.00,2021-10-19 15:59:58,2021-10-19 15:59:48,27,1,XO,MP,16,0.4639,864.27,0.26973,0.000549,-0.122301,4.28636,3.61674,0.4639,0.0,2021-10-19 16:16:38,2021-10-19 16:18:00
2000825,2021-10-19 16:18:00,21400,TSLA,2024-01-19 00:00:00,1700.0,P,A,TSLA 240119P01700000,879.50,896.55,2021-10-19 15:59:58,2021-10-19 15:59:58,10,65,AO,XO,0,0.4353,864.27,-0.76402,0.000569,-0.097444,3.92524,-18.19406,0.4353,0.0,2021-10-19 16:16:38,2021-10-19 16:18:00
2000826,2021-10-19 16:18:00,21400,TSLA,2024-01-19 00:00:00,1725.0,C,A,TSLA 240119C01725000,68.00,76.00,2021-10-19 15:59:58,2021-10-19 15:59:58,25,17,XO,BT,60,0.4682,864.27,0.26678,0.000541,-0.122713,4.26275,3.57122,0.4682,0.0,2021-10-19 16:16:38,2021-10-19 16:18:00


# Pre-processing and Cleaning

In [ ]:
# creating target column
tsla['price'] = (tsla['price_bid'] + tsla['price_ask'])  / 2

In [ ]:
# dropping non-numeric and constant columns 
dropped_cols = ['stock_id', 'stock_symbol', 'style', 'implied_yield', 'symbol',
                'exchange_bid', 'exchange_ask', 'price_bid', 'price_ask', 't_date',
                'expiration_date', 'date_bid', 'date_ask', 'calc_date', 'dump_time']
tsla.drop(dropped_cols, axis=1, inplace=True)

In [ ]:
# splitting dataset into calls and puts
calls = tsla[tsla['call_put'] == 'C'].drop('call_put', axis=1)
puts = tsla[tsla['call_put'] == 'P'].drop('call_put', axis=1)

In [ ]:
calls

,strike,size_bid,size_ask,volume,implied_volatility,price_opt,delta,gamma,theta,vega,rho,pre_iv,price
0,50.0,0,0,0,-1.0000,870.11,0.00000,0.000000,0.000000,0.00000,0.00000,-1.0000,0.000
2,100.0,0,0,0,-1.0000,870.11,0.00000,0.000000,0.000000,0.00000,0.00000,-1.0000,0.000
4,150.0,0,0,0,-1.0000,870.11,0.00000,0.000000,0.000000,0.00000,0.00000,-1.0000,0.000
6,200.0,0,0,0,-1.0000,870.11,0.00000,0.000000,0.000000,0.00000,0.00000,-1.0000,0.000
8,250.0,0,0,0,-1.0000,870.11,0.00000,0.000000,0.000000,0.00000,0.00000,-1.0000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000818,1600.0,1,256,146,0.4745,864.27,0.30880,0.000572,-0.133333,4.56806,4.04002,0.4745,87.500
2000820,1650.0,29,20,0,0.4670,864.27,0.28690,0.000562,-0.126888,4.41730,3.81019,0.4670,78.775
2000822,1675.0,25,17,0,0.4675,864.27,0.28014,0.000555,-0.125580,4.36716,3.72998,0.4675,76.500
2000824,1700.0,27,1,16,0.4639,864.27,0.26973,0.000549,-0.122301,4.28636,3.61674,0.4639,72.525


# Feature Engineering

In [ ]:
X = calls.drop('price', axis=1)
y = calls['price']

afreg = AutoFeatRegressor(verbose=1, feateng_steps=1)
X_new = afreg.fit_transform(X, y)
print('New features: ', afreg.new_feat_cols_)

[AutoFeat] The 1 step feature engineering process could generate up to 84 features.
[AutoFeat] With 1000414 data points this new feature matrix would use about 0.34 gb of space.
[feateng] Step 1: transformation of original features
[feateng] Generated 44 transformed features from 12 original features - done.
[feateng] Generated altogether 55 new features in 1 steps
[feateng] Removing correlated features, as well as additions at the highest level
[feateng] Generated a total of 29 additional features
[featsel] Scaling data...done.
[featsel] Feature selection run 1/5
[featsel] Feature selection run 2/5
[featsel] Feature selection run 3/5
[featsel] Feature selection run 4/5
[featsel] Feature selection run 5/5
[featsel] 33 features after 5 feature selection runs
[featsel] 27 features after correlation filtering
[featsel] 25 features after noise filtering
[AutoFeat] Computing 13 new features.
[AutoFeat]    13/   13 new features ...done.
[AutoFeat] Final dataframe with 25 feature columns (13 

In [ ]:
X_new

,strike,size_bid,size_ask,volume,implied_volatility,price_opt,delta,gamma,theta,vega,rho,pre_iv,rho**3,vega**3,1/strike,1/pre_iv,theta**3,pre_iv**2,pre_iv**3,strike**3,size_bid**3,log(strike),sqrt(volume),1/implied_volatility,implied_volatility**3
0,50.0,0.0,0.0,0.0,-1.0000,870.11,0.00000,0.000000,0.000000,0.00000,0.00000,-1.0000,0.000000,0.000000,0.020000,-1.000000,0.000000,1.000000,-1.000000,1.250000e+05,0.0,3.912023,0.000000,-1.000000,-1.000000
1,100.0,0.0,0.0,0.0,-1.0000,870.11,0.00000,0.000000,0.000000,0.00000,0.00000,-1.0000,0.000000,0.000000,0.010000,-1.000000,0.000000,1.000000,-1.000000,1.000000e+06,0.0,4.605170,0.000000,-1.000000,-1.000000
2,150.0,0.0,0.0,0.0,-1.0000,870.11,0.00000,0.000000,0.000000,0.00000,0.00000,-1.0000,0.000000,0.000000,0.006667,-1.000000,0.000000,1.000000,-1.000000,3.375000e+06,0.0,5.010635,0.000000,-1.000000,-1.000000
3,200.0,0.0,0.0,0.0,-1.0000,870.11,0.00000,0.000000,0.000000,0.00000,0.00000,-1.0000,0.000000,0.000000,0.005000,-1.000000,0.000000,1.000000,-1.000000,8.000000e+06,0.0,5.298317,0.000000,-1.000000,-1.000000
4,250.0,0.0,0.0,0.0,-1.0000,870.11,0.00000,0.000000,0.000000,0.00000,0.00000,-1.0000,0.000000,0.000000,0.004000,-1.000000,0.000000,1.000000,-1.000000,1.562500e+07,0.0,5.521461,0.000000,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000409,1600.0,1.0,256.0,146.0,0.4745,864.27,0.30880,0.000572,-0.133333,4.56806,4.04002,0.4745,65.940243,95.322494,0.000625,2.107482,-0.002370,0.225150,0.106834,4.096000e+09,1.0,7.377759,12.083046,2.107482,0.106834
1000410,1650.0,29.0,20.0,0.0,0.4670,864.27,0.28690,0.000562,-0.126888,4.41730,3.81019,0.4670,55.314616,86.192740,0.000606,2.141328,-0.002043,0.218089,0.101848,4.492125e+09,24389.0,7.408531,0.000000,2.141328,0.101848
1000411,1675.0,25.0,17.0,0.0,0.4675,864.27,0.28014,0.000555,-0.125580,4.36716,3.72998,0.4675,51.894282,83.290853,0.000597,2.139037,-0.001980,0.218556,0.102175,4.699422e+09,15625.0,7.423568,0.000000,2.139037,0.102175
1000412,1700.0,27.0,1.0,16.0,0.4639,864.27,0.26973,0.000549,-0.122301,4.28636,3.61674,0.4639,47.309882,78.752787,0.000588,2.155637,-0.001829,0.215203,0.099833,4.913000e+09,19683.0,7.438384,4.000000,2.155637,0.099833


# Saving the data

In [ ]:
X_new.to_csv(path + 'calls_data.csv', index=False)
y.to_csv(path + 'calls_target.csv', index=False)

# Repeating Feature Engineering for Puts

In [ ]:
X = puts.drop('price', axis=1)
y = puts['price']

afreg = AutoFeatRegressor(verbose=1, feateng_steps=1)
X_new = afreg.fit_transform(X, y)
print('New features: ', afreg.new_feat_cols_)

[AutoFeat] The 1 step feature engineering process could generate up to 84 features.
[AutoFeat] With 1000414 data points this new feature matrix would use about 0.34 gb of space.
[feateng] Step 1: transformation of original features
[feateng] Generated 42 transformed features from 12 original features - done.
[feateng] Generated altogether 53 new features in 1 steps
[feateng] Removing correlated features, as well as additions at the highest level
[feateng] Generated a total of 31 additional features
[featsel] Scaling data...done.
[featsel] Feature selection run 1/5
[featsel] Feature selection run 2/5
[featsel] Feature selection run 3/5
[featsel] Feature selection run 4/5
[featsel] Feature selection run 5/5
[featsel] 36 features after 5 feature selection runs
[featsel] 28 features after correlation filtering
[featsel] 27 features after noise filtering
[AutoFeat] Computing 15 new features.
[AutoFeat]    15/   15 new features ...done.
[AutoFeat] Final dataframe with 27 feature columns (15 

In [ ]:
X_new.to_csv(path + 'puts_data.csv', index=False)
y.to_csv(path + 'puts_target.csv', index=False)